<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/Imocap_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
pip install transformers datasets evaluate

In [89]:
from huggingface_hub import notebook_login

notebook_login()

# **Loading Base wav2vec2 model**

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [90]:
%%capture
!pip install gdown

In [91]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [92]:
!gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
time.sleep(5)
!unzip /content/IEMOCAP_splits.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
To: /content/IEMOCAP_splits.zip
100% 817k/817k [00:00<00:00, 29.5MB/s]
Archive:  /content/IEMOCAP_splits.zip
replace iemocap/session5/test.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [93]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session1/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/iemocap/session1/test.json",'r') as f:
    test_data = json.load(f)

In [99]:
import json

# Load the JSON data
with open("/content/iemocap/session1/train.json", "r") as file:
    data = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data, file, indent=4)

print("File paths updated successfully.")


File paths updated successfully.


In [100]:
train_data =data

In [101]:
train_data

{'Ses02F_impro03_F000': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav',
  'length': 4.0475736961451245,
  'emo': 'hap'},
 'Ses02F_impro03_F001': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F001.wav',
  'length': 2.119954648526077,
  'emo': 'hap'},
 'Ses02F_impro03_F004': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F004.wav',
  'length': 2.0,
  'emo': 'hap'},
 'Ses02F_impro03_F009': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F009.wav',
  'length': 12.08702947845805,
  'emo': 'hap'},
 'Ses02F_impro03_F011': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F011.wav',
  'length': 4.207210884353741,
  'emo': 'hap'},
 'Ses02F_impro03_F012': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F012.wav',
  'length': 1.420907029478458,
  'emo': 'hap'},
 'Ses02F_impro03_F013': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F013.wav',
  'length': 6.3349659863945575,
  'emo': 'hap'

In [102]:
test_data=data

In [103]:
test_data

{'Ses02F_impro03_F000': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav',
  'length': 4.0475736961451245,
  'emo': 'hap'},
 'Ses02F_impro03_F001': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F001.wav',
  'length': 2.119954648526077,
  'emo': 'hap'},
 'Ses02F_impro03_F004': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F004.wav',
  'length': 2.0,
  'emo': 'hap'},
 'Ses02F_impro03_F009': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F009.wav',
  'length': 12.08702947845805,
  'emo': 'hap'},
 'Ses02F_impro03_F011': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F011.wav',
  'length': 4.207210884353741,
  'emo': 'hap'},
 'Ses02F_impro03_F012': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F012.wav',
  'length': 1.420907029478458,
  'emo': 'hap'},
 'Ses02F_impro03_F013': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F013.wav',
  'length': 6.3349659863945575,
  'emo': 'hap'

In [104]:
import json

# Load the JSON data
with open("/content/iemocap/session1/test.json", "r") as file:
    data = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [105]:
 test_data=data

In [106]:
test_data

{'Ses01F_script02_2_F000': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_script02_2_F010': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F010.wav',
  'length': 3.6105215419501135,
  'emo': 'ang'},
 'Ses01F_script02_2_F015': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_scrip

In [110]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session2/train.json",'r') as f:
    train_data2 = json.load(f)

with open("/content/iemocap/session2/test.json",'r') as f:
    test_data2 = json.load(f)

In [113]:
import json

# Load the JSON data
with open("/content/iemocap/session2/train.json", "r") as file:
    data2 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data2.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data2, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [114]:
train_data2=data2

In [115]:
train_data2

{'Ses01F_script02_2_F000': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_script02_2_F010': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F010.wav',
  'length': 3.6105215419501135,
  'emo': 'ang'},
 'Ses01F_script02_2_F015': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_scrip

In [116]:
import json

# Load the JSON data
with open("/content/iemocap/session2/test.json", "r") as file:
    data2 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data2.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data2, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [117]:
test_data2=data2

In [118]:
test_data2

{'Ses02F_impro03_F000': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav',
  'length': 4.0475736961451245,
  'emo': 'hap'},
 'Ses02F_impro03_F001': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F001.wav',
  'length': 2.119954648526077,
  'emo': 'hap'},
 'Ses02F_impro03_F004': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F004.wav',
  'length': 2.0,
  'emo': 'hap'},
 'Ses02F_impro03_F009': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F009.wav',
  'length': 12.08702947845805,
  'emo': 'hap'},
 'Ses02F_impro03_F011': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F011.wav',
  'length': 4.207210884353741,
  'emo': 'hap'},
 'Ses02F_impro03_F012': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F012.wav',
  'length': 1.420907029478458,
  'emo': 'hap'},
 'Ses02F_impro03_F013': {'wav': 'Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F013.wav',
  'length': 6.3349659863945575,
  'emo': 'hap'

In [119]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session3/train.json",'r') as f:
    train_data3 = json.load(f)

with open("/content/iemocap/session3/test.json",'r') as f:
    test_data3 = json.load(f)

In [120]:
import json

# Load the JSON data
with open("/content/iemocap/session3/train.json", "r") as file:
    data3 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data3.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data3, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [121]:
train_data3=data3

In [122]:
train_data3

{'Ses01F_script02_2_F000': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_script02_2_F010': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F010.wav',
  'length': 3.6105215419501135,
  'emo': 'ang'},
 'Ses01F_script02_2_F015': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_scrip

In [123]:
import json

# Load the JSON data
with open("/content/iemocap/session3/test.json", "r") as file:
    data3 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data3.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data3, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [124]:
test_data3=data3

In [125]:
test_data3

{'Ses03F_script02_1_F000': {'wav': 'Session3/sentences/wav/Ses03F_script02_1/Ses03F_script02_1_F000.wav',
  'length': 1.2163265306122448,
  'emo': 'sad'},
 'Ses03F_script02_1_F003': {'wav': 'Session3/sentences/wav/Ses03F_script02_1/Ses03F_script02_1_F003.wav',
  'length': 7.591292517006803,
  'emo': 'ang'},
 'Ses03F_script02_1_F010': {'wav': 'Session3/sentences/wav/Ses03F_script02_1/Ses03F_script02_1_F010.wav',
  'length': 2.732834467120181,
  'emo': 'sad'},
 'Ses03F_script02_1_F011': {'wav': 'Session3/sentences/wav/Ses03F_script02_1/Ses03F_script02_1_F011.wav',
  'length': 2.7513832199546484,
  'emo': 'sad'},
 'Ses03F_script02_1_F012': {'wav': 'Session3/sentences/wav/Ses03F_script02_1/Ses03F_script02_1_F012.wav',
  'length': 6.595011337868481,
  'emo': 'ang'},
 'Ses03F_script02_1_M000': {'wav': 'Session3/sentences/wav/Ses03F_script02_1/Ses03F_script02_1_M000.wav',
  'length': 8.469977324263038,
  'emo': 'hap'},
 'Ses03F_script02_1_M001': {'wav': 'Session3/sentences/wav/Ses03F_script02

In [126]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session4/train.json",'r') as f:
    train_data4 = json.load(f)

with open("/content/iemocap/session4/test.json",'r') as f:
    test_data4 = json.load(f)

In [127]:
import json

# Load the JSON data
with open("/content/iemocap/session4/train.json", "r") as file:
    data4 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data4.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data4, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [128]:
train_data4=data4

In [129]:
train_data4

{'Ses01F_script02_2_F000': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_script02_2_F010': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F010.wav',
  'length': 3.6105215419501135,
  'emo': 'ang'},
 'Ses01F_script02_2_F015': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_scrip

In [130]:
import json

# Load the JSON data
with open("/content/iemocap/session4/test.json", "r") as file:
    data4 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data4.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data4, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [131]:
test_data4=data4

In [132]:
test_data4

{'Ses04F_script03_1_F005': {'wav': 'Session4/sentences/wav/Ses04F_script03_1/Ses04F_script03_1_F005.wav',
  'length': 4.555102040816327,
  'emo': 'hap'},
 'Ses04F_script03_1_F006': {'wav': 'Session4/sentences/wav/Ses04F_script03_1/Ses04F_script03_1_F006.wav',
  'length': 2.039954648526077,
  'emo': 'hap'},
 'Ses04F_script03_1_F007': {'wav': 'Session4/sentences/wav/Ses04F_script03_1/Ses04F_script03_1_F007.wav',
  'length': 2.54,
  'emo': 'hap'},
 'Ses04F_script03_1_F009': {'wav': 'Session4/sentences/wav/Ses04F_script03_1/Ses04F_script03_1_F009.wav',
  'length': 2.68,
  'emo': 'hap'},
 'Ses04F_script03_1_F011': {'wav': 'Session4/sentences/wav/Ses04F_script03_1/Ses04F_script03_1_F011.wav',
  'length': 2.8899773242630387,
  'emo': 'hap'},
 'Ses04F_script03_1_F012': {'wav': 'Session4/sentences/wav/Ses04F_script03_1/Ses04F_script03_1_F012.wav',
  'length': 1.4499773242630385,
  'emo': 'neu'},
 'Ses04F_script03_1_F014': {'wav': 'Session4/sentences/wav/Ses04F_script03_1/Ses04F_script03_1_F014.

In [133]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session5/train.json",'r') as f:
    train_data5 = json.load(f)

with open("/content/iemocap/session5/test.json",'r') as f:
    test_data5 = json.load(f)

In [135]:
import json

# Load the JSON data
with open("/content/iemocap/session5/train.json", "r") as file:
    data5 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data5.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data5, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [136]:
train_data5=data5

In [137]:
train_data5

{'Ses01F_script02_2_F000': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_script02_2_F010': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F010.wav',
  'length': 3.6105215419501135,
  'emo': 'ang'},
 'Ses01F_script02_2_F015': {'wav': 'Session1/sentences/wav/Ses01F_script02_2/Ses01F_scrip

In [138]:
import json

# Load the JSON data
with open("/content/iemocap/session5/test.json", "r") as file:
    data5 = json.load(file)

# Define the path prefix to remove
path_prefix = "/m/teamwork/t40511_asr/c/IEMOCAP_full_release/"

# Update each entry in the JSON
for file_id, entry in data5.items():
    if entry["wav"].startswith(path_prefix):
        # Remove the prefix from the wav path
        entry["wav"] = entry["wav"][len(path_prefix):]

# Save the updated data back to JSON
with open("updated_dataset.json", "w") as file:
    json.dump(data5, file, indent=4)

print("File paths updated successfully.")

File paths updated successfully.


In [139]:
test_data=data5

In [140]:
test_data

{'Ses05M_script01_3_F000': {'wav': 'Session5/sentences/wav/Ses05M_script01_3/Ses05M_script01_3_F000.wav',
  'length': 2.9499773242630387,
  'emo': 'neu'},
 'Ses05M_script01_3_F001': {'wav': 'Session5/sentences/wav/Ses05M_script01_3/Ses05M_script01_3_F001.wav',
  'length': 7.289977324263038,
  'emo': 'hap'},
 'Ses05M_script01_3_F003': {'wav': 'Session5/sentences/wav/Ses05M_script01_3/Ses05M_script01_3_F003.wav',
  'length': 3.7300226757369614,
  'emo': 'neu'},
 'Ses05M_script01_3_F006': {'wav': 'Session5/sentences/wav/Ses05M_script01_3/Ses05M_script01_3_F006.wav',
  'length': 3.5500226757369613,
  'emo': 'neu'},
 'Ses05M_script01_3_F007': {'wav': 'Session5/sentences/wav/Ses05M_script01_3/Ses05M_script01_3_F007.wav',
  'length': 5.150022675736961,
  'emo': 'neu'},
 'Ses05M_script01_3_F009': {'wav': 'Session5/sentences/wav/Ses05M_script01_3/Ses05M_script01_3_F009.wav',
  'length': 3.0700226757369613,
  'emo': 'hap'},
 'Ses05M_script01_3_F010': {'wav': 'Session5/sentences/wav/Ses05M_script

In [141]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [142]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [143]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1241
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1241
    })
})

In [144]:
import os

def add_base_path(example):
    # Assuming the original paths are relative to /content/iemocap/
    example['wav'] = os.path.join('/content/iemocap/', example['wav'])
    return example

dataset = dataset.map(add_base_path, batched=False)

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

In [145]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [146]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/iemocap/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav'